# 텍스트 데이터 증강
  -보통 Data Augmentation은 학습데이터가 부족해서 데이터를 늘리거나 불균형 데이터의 레이블들의 균형을 잡아주기 위하여 진행하곤 한다.

여러가지 텍스트 데이터 증강하는 방법이 있지만 Bert_augmentation을 진행해서 데이터를 늘릴려고 한다.

참고 사이트 https://github.com/kyle-bong/K-TACC(bert_augmentation으로 데이터를 증강하는 방법을 자세히 다루고 있다.)

In [15]:
cd "/content/drive/MyDrive"

/content/drive/MyDrive


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
!git clone https://github.com/kyle-bong/K-TACC.git

Cloning into 'K-TACC'...
remote: Enumerating objects: 518, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 518 (delta 70), reused 70 (delta 70), pack-reused 380 (from 1)
Receiving objects: 100% (518/518), 39.58 MiB | 14.62 MiB/s, done.
Resolving deltas: 100% (356/356), done.
Updating files: 100% (36/36), done.


In [17]:
import pandas as pd
from K_TACC.utils.BertAugmentation import BertAugmentation
from K_TACC.utils.adverb_augmentation import AdverbAugmentation
import transformers

In [18]:
BERT_aug = BertAugmentation()
random_masking_replacement = BERT_aug.random_masking_replacement
random_masking_insertion = BERT_aug.random_masking_insertion

tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [19]:
adverb_aug = AdverbAugmentation()
adverb_gloss_replacement = adverb_aug.adverb_gloss_replacement

# BERT augmentation

In [20]:
sentence = "아버지가 오늘 아침에 부엌에 있었다가 갑자기 일어나시더니 슬그머니 안방으로 들어가셨다."
ratio = 0.3

In [21]:
sentence = "저녁에 배가 너무 고파서 빵과 우유를 먹고 난 후 배탈이 났다."
ratio = 0.3

In [22]:
print('random_masking_replacement:', random_masking_replacement(sentence, ratio))
print('random_insertion_replacement:', random_masking_insertion(sentence, ratio))

random_masking_replacement: 저녁에는 너무 고파서 빵과 우유를 먹고 난 후 배탈이 났다.
random_insertion_replacement: 저녁에 배가 너무도 많이 고파서, 빵과 우유를 먹고 난 후 배탈이 났다.


# Adverb augmentation

In [23]:
from quickspacer import Spacer
spacer = Spacer()
spacer.space(['생각할겨를도없이빨리'])

['생각할 겨를도 없이 빨리']

In [24]:
sentence = "아버지가 부엌에 있었다가 갑자기 일어나시더니 슬그머니 안방으로 들어가셨다."

In [25]:
adverb_gloss_replacement(sentence)

'아버지가 부엌에 있었다가 생각할 겨를도 없이 빨리 일어나시더니 슬그머니 안방으로 들어가셨다.'

In [26]:
# Then load it from Google Drive
dataset = pd.read_csv('/content/drive/MyDrive/ChatbotData2.csv', encoding='utf-8-sig')

In [27]:
Q_random_masking_replacement = dataset['Q'].apply(lambda x: random_masking_replacement(x, ratio))

In [28]:
A_random_masking_replacement = dataset['A'].apply(lambda x: random_masking_replacement(x, ratio))

In [29]:
Q_random_masking_insertion = dataset['Q'].apply(lambda x: random_masking_insertion(x, ratio))

In [30]:
A_random_masking_insertion = dataset['A'].apply(lambda x: random_masking_insertion(x, ratio))

In [31]:
Q_adverb_gloss_replacement = dataset['Q'].apply(lambda x: adverb_gloss_replacement(x))

In [32]:
A_adverb_gloss_replacement = dataset['A'].apply(lambda x: adverb_gloss_replacement(x))

In [44]:
q_list =Q_random_masking_replacement.tolist()
a_list =A_random_masking_replacement.tolist()

In [45]:
new_dataset_1 = pd.DataFrame({
    'Q': q_list,
    'A': a_list
})

In [47]:
q_list2 =Q_random_masking_insertion.tolist()
a_list2 =A_random_masking_insertion.tolist()

In [48]:
new_dataset_2 = pd.DataFrame({
    'Q':q_list2,
    'A':a_list2
})

In [49]:
q_list3 = Q_adverb_gloss_replacement.tolist()
a_list3 = A_adverb_gloss_replacement.tolist()

In [50]:
new_dataset_3 = pd.DataFrame({
    'Q':q_list3,
    'A':a_list3
})

In [51]:
datasets = pd.concat([new_dataset_1, new_dataset_2, new_dataset_3], ignore_index=True)

In [53]:
datasets.to_csv("augmentation.csv",encoding='utf-8-sig')